In [1]:
import os
from PIL import Image
import numpy as np
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import json
from tensorflow.keras.utils import to_categorical

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:

def list_subdirectories(directory_path):
    try:
        subdirectories = [f.path for f in os.scandir(directory_path) if f.is_dir()]
        return subdirectories
    except Exception as e:
        return str(e)

def list_all_subdirectories_with_images(parent_directory):
    subdirectories = list_subdirectories(parent_directory)
    XT = []  # List to store all images
    YT = []  # List to store labels
    YT_count = []
    #print(subdirectories)
    
    label= 0
    length_list = []
    for s in subdirectories:
        
        sub = [f.path for f in os.scandir(s) if f.is_dir()]
        #print(sub)
        for f in sub:
            count = 0
            for img_path in os.listdir(f):
            
                if img_path.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    img = Image.open(os.path.join(f, img_path))
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                    img = img.resize((32, 32), Image.ANTIALIAS)
                    img = np.array(img).astype('float32') / 255.0
                    XT.append(img)
                    count = count + 1
                    YT.append(label)
                #print(np.shape(lst))
            length_list.append(count)
            YT_count.append(label)
        label=label+1    
                
        
    
    return XT, YT, YT_count, length_list


# Example usage:
directory_path = r'E:\invasive-aquatic-species-data'
images_list, YT, YT_count, lengths_list = list_all_subdirectories_with_images(directory_path)

import random
combined_data = list(zip(images_list, YT))
random.shuffle(combined_data)
images_list, YT = zip(*combined_data)
yt = to_categorical(YT)
ytc = to_categorical(YT_count)


k = int(.7 * len(lengths_list))


# Define the number of samples for testing
num_test_samples = np.sum(lengths_list[k:])
# Split the lists
X_test = images_list[:num_test_samples]
Y_test = yt[:num_test_samples]
X_train = images_list[num_test_samples:]
Y_train = yt[num_test_samples:]
np.shape(Y_train)

C:\Users\shaif\AppData\Local\Temp\ipykernel_12344\3042067344.py:29: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((32, 32), Image.ANTIALIAS)


(80644, 2)

In [3]:
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(2, activation='softmax'))  # Change units to match the number of classes
optimizer = Adam()  # Define optimizer separately
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# Assuming X_train and Y_train are defined previously
model.fit(np.array(X_train).astype('float32'), Y_train, batch_size=32, epochs=20, validation_split=0.2)

Epoch 1/20
2017/2017 [==============================] - 102s 43ms/step - loss: 0.1994 - accuracy: 0.9255 - val_loss: 0.1851 - val_accuracy: 0.9041
Epoch 2/20
2017/2017 [==============================] - 88s 44ms/step - loss: 0.1687 - accuracy: 0.9316 - val_loss: 0.3634 - val_accuracy: 0.9076
Epoch 3/20
2017/2017 [==============================] - 93s 46ms/step - loss: 0.1662 - accuracy: 0.9349 - val_loss: 0.2136 - val_accuracy: 0.9109
Epoch 4/20
2017/2017 [==============================] - 110s 55ms/step - loss: 0.1932 - accuracy: 0.9271 - val_loss: 0.1508 - val_accuracy: 0.9353
Epoch 5/20
2017/2017 [==============================] - 177s 88ms/step - loss: 0.1596 - accuracy: 0.9337 - val_loss: 0.1320 - val_accuracy: 0.9447
Epoch 6/20
2017/2017 [==============================] - 170s 84ms/step - loss: 0.1469 - accuracy: 0.9399 - val_loss: 0.1499 - val_accuracy: 0.9373
Epoch 7/20
2017/2017 [==============================] - 158s 79ms/step - loss: 0.1754 - accuracy: 0.9313 - val_loss: 0.1

In [4]:

from sklearn.metrics import accuracy_score

predictions = model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(Y_test,axis = 1),np.argmax(predictions,axis = 1))
print("Test Accuracy:", accuracy)
from sklearn.metrics import accuracy_score, balanced_accuracy_score

# Assuming you have ground truth labels in Y_test and predicted labels in predictions
ground_truth_labels = np.argmax(Y_test, axis=1)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate balanced accuracy
balanced_accuracy = balanced_accuracy_score(ground_truth_labels, predicted_labels)

# Print the balanced accuracy
print("Balanced Accuracy:", balanced_accuracy)


Test Accuracy: 0.9498195619711299
Balanced Accuracy: 0.8853120171640378
